In [1]:
from imutils import paths
import numpy as np
import imutils
import pickle
import cv2
import os, sys

sys.path.append('../utilities')
from alignFaces import *

In [2]:
data_params = {
    "dataset":"../../facial_recognition_data/faces_dataset/", # path to input directory of images
    "embeddings":"../../output/embeddings.pickle", # path to output of database of facial embeddings
    "detector":"../../facial_detection/models/", # path to OpenCV's deep learning face detector
    "embedding_model":"../models/openface.nn4.small2.v1.t7", # path to OpenCV's deep learning face embedding model
    "confidence":0.5 # min probability to filter weak detections
}

In [3]:
# load our serialized face detector from disk
print("[INFO] loading face detector...")
protoPath = os.path.sep.join([data_params["detector"], "deploy.prototxt"])
modelPath = os.path.sep.join([data_params["detector"], "res10_300x300_ssd_iter_140000.caffemodel"])
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

# load our serialized face embedding model from disk
print("[INFO] loading face recognizer...")
embedder = cv2.dnn.readNetFromTorch(data_params["embedding_model"])

[INFO] loading face detector...
[INFO] loading face recognizer...


In [4]:
# grab the paths to the input images in our dataset
print("[INFO] quantifying faces...")
imagePaths = list(paths.list_images(data_params["dataset"]))
print(*imagePaths, sep="\n")
# initialize our lists of extracted facial embeddings and
# corresponding people names
knownEmbeddings = []
knownNames = []

# initialize the total number of faces processed
total = 0

[INFO] quantifying faces...
../../facial_recognition_data/faces_dataset/andrew\andrew_frame0.jpg
../../facial_recognition_data/faces_dataset/andrew\andrew_frame100.jpg
../../facial_recognition_data/faces_dataset/andrew\andrew_frame1000.jpg
../../facial_recognition_data/faces_dataset/andrew\andrew_frame1020.jpg
../../facial_recognition_data/faces_dataset/andrew\andrew_frame1040.jpg
../../facial_recognition_data/faces_dataset/andrew\andrew_frame1060.jpg
../../facial_recognition_data/faces_dataset/andrew\andrew_frame1080.jpg
../../facial_recognition_data/faces_dataset/andrew\andrew_frame1100.jpg
../../facial_recognition_data/faces_dataset/andrew\andrew_frame1120.jpg
../../facial_recognition_data/faces_dataset/andrew\andrew_frame1140.jpg
../../facial_recognition_data/faces_dataset/andrew\andrew_frame1160.jpg
../../facial_recognition_data/faces_dataset/andrew\andrew_frame1180.jpg
../../facial_recognition_data/faces_dataset/andrew\andrew_frame120.jpg
../../facial_recognition_data/faces_datas

In [5]:
# loop over the image paths
af = AlignFaces()
for (i, imagePath) in enumerate(imagePaths):
    print("[INFO] processing image {}/{}".format(i + 1, len(imagePaths)))
    # The right side of the last occurance of a '/' character is gotten from the path.
    # Then the string is split based on the '\' character which prepends the image name.
    # The left side of the split, representing the directory/person name is saved as name.
    name = imagePath[imagePath.rindex('/')+1:].split('\\')[0]
    
    image = cv2.imread(imagePath) # Load the image
    image = af.align_faces(image) # Align the image.
    (h, w) = image.shape[:2] # Grab the image dimensions.
    
    # construct a blob from the image
    imageBlob = cv2.dnn.blobFromImage(
        cv2.resize(image, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0), swapRB=False, crop=False)

    # apply OpenCV's deep learning-based face detector to localize faces in the input image
    detector.setInput(imageBlob)
    detections = detector.forward()
    
    # If at least one face was found, then we continue. Else, we begin the loop once more.
    if len(detections) > 0:
        if len(detections) > 1:
            print("[INFO] {} detections found in image.".format(len(detections)))
        
        # We assume that each image has only ONE face, so find the bounding box with the largest probability
        i = np.argmax(detections[0, 0, :, 2])
        confidence = detections[0, 0, i, 2]

        # The detection with the largest probability is compared against our minimum probability
        # This helps filter out weak detections
        if confidence > data_params["confidence"]:
            # The (x, y)-coordinates of the bounding box for the face are calculated
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # The face region of interest (ROI) is extracted
            face = image[startY:endY, startX:endX]
            (fH, fW) = face.shape[:2]

            # If the face's width and height are too small, we begin the loop once more.
            if fW < 20 or fH < 20:
                continue
            
            # A blob is constructed for the face ROI. This is then passed
            # through our face embedding model to obtain the 128-d quantification of the face
            faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,
                (96, 96), (0, 0, 0), swapRB=True, crop=False)
            embedder.setInput(faceBlob)
            vec = embedder.forward()

            # add the name of the person + corresponding face
            # embedding to their respective lists
            knownNames.append(name)
            knownEmbeddings.append(vec.flatten())
            total += 1

[INFO] processing image 1/280
[INFO] processing image 2/280
[INFO] processing image 3/280
[INFO] processing image 4/280
[INFO] processing image 5/280
[INFO] processing image 6/280
[INFO] processing image 7/280
[INFO] processing image 8/280
[INFO] processing image 9/280
[INFO] processing image 10/280
[INFO] processing image 11/280
[INFO] processing image 12/280
[INFO] processing image 13/280
[INFO] processing image 14/280
[INFO] processing image 15/280
[INFO] processing image 16/280
[INFO] processing image 17/280
[INFO] processing image 18/280
[INFO] processing image 19/280
[INFO] processing image 20/280
[INFO] processing image 21/280
[INFO] processing image 22/280
[INFO] processing image 23/280
[INFO] processing image 24/280
[INFO] processing image 25/280
[INFO] processing image 26/280
[INFO] processing image 27/280
[INFO] processing image 28/280
[INFO] processing image 29/280
[INFO] processing image 30/280
[INFO] processing image 31/280
[INFO] processing image 32/280
[INFO] processing

[INFO] processing image 231/280
[INFO] processing image 232/280
[INFO] processing image 233/280
[INFO] processing image 234/280
[INFO] processing image 235/280
[INFO] processing image 236/280
[INFO] processing image 237/280
[INFO] processing image 238/280
[INFO] processing image 239/280
[INFO] processing image 240/280
[INFO] processing image 241/280
[INFO] processing image 242/280
[INFO] processing image 243/280
[INFO] processing image 244/280
[INFO] processing image 245/280
[INFO] processing image 246/280
[INFO] no detections found in image. Returning original.
[INFO] processing image 247/280
[INFO] processing image 248/280
[INFO] processing image 249/280
[INFO] processing image 250/280
[INFO] processing image 251/280
[INFO] processing image 252/280
[INFO] processing image 253/280
[INFO] processing image 254/280
[INFO] processing image 255/280
[INFO] processing image 256/280
[INFO] processing image 257/280
[INFO] processing image 258/280
[INFO] processing image 259/280
[INFO] processi

In [6]:
# dump the facial embeddings + names to disk
print("[INFO] serializing {} encodings...".format(total))
data = {"embeddings": knownEmbeddings, "names": knownNames}
f = open(data_params["embeddings"], "wb")
f.write(pickle.dumps(data))
f.close()

[INFO] serializing 216 encodings...
